In [126]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [127]:
training_df = pd.read_csv('nn.csv', index_col=0)
training_df.drop(columns=[training_df.columns[0], training_df.columns[1], training_df.columns[2], 
                          training_df.columns[3], training_df.columns[len(training_df.columns)-1]], 
                 axis=1,  inplace=True)
cols_at_end = ['Analyst']
training_df = training_df[[c for c in training_df if c not in cols_at_end] 
        + [c for c in cols_at_end if c in training_df]]

# Fill in missing
def fillMissing(df, feature, method):
  if method == "mode":
    df[feature] = df[feature].fillna(df[feature].mode()[0])
  elif method == "median":
    df[feature] = df[feature].fillna(df[feature].median())
  else:
    df[feature] = df[feature].fillna(df[feature].mean())

features_missing = training_df.columns[training_df.isna().any()]
copy_df = training_df.copy()
for feature in features_missing:
  fillMissing(training_df, feature = feature, method = "mean")
training_df = training_df.round(2)

In [128]:
def f(row):
    if row['Analyst'] == 'Strong Buy' or row['Analyst'] == 'Buy':
        val = 'Buy'
    elif row['Analyst'] == 'Strong Sell' or row['Analyst'] == 'Sell':
        val = 'Sell'
    elif row['Analyst'] == 'Hold':
        val = 'Hold'
    else:
        val = 'Invalid'
    return val

training_df['Prediction'] = training_df.apply(f, axis=1)

In [129]:
X = training_df[['headline_polarity', 'convo_polarity', 'forwardEps','bookValue', 'heldPercentInstitutions', 
        'shortRatio', 'shortPercentOfFloat']]
y = training_df['Prediction']

encoder = LabelEncoder()
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [130]:
training_df['Prediction'].value_counts()

Buy     3849
Hold     410
Sell      66
Name: Prediction, dtype: int64

In [144]:
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import RMSprop
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(7,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    batch_size=16,
    epochs=50,
    verbose=0,
    validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.42268308997154236
Test accuracy: 0.8901734352111816
